In [3]:
#@title Setup Environment
#@test {"output": "ignore"}

import glob

print 'Copying checkpoints and example MIDI from GCS. This will take a few minutes...'
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/* /content/

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print 'Importing libraries and defining some helper functions...'
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print 'Start Seq Reconstruction'
  play(note_sequences[0])
  print 'End Seq Reconstruction'
  play(note_sequences[-1])
  print 'Mean Sequence'
  play(note_sequences[num_steps // 2])
  print 'Start -> End Interpolation'
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print 'Done'

Copying checkpoints and example MIDI from GCS. This will take a few minutes...
Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 1.6MB 49.4MB/s 
     |████████████████████████████████| 2.9MB 51.6MB/s 
     |████████████████████████████████| 2.4MB 65.7MB/s 
     |████████████████████████████████| 983kB 48.3MB/s 
     |██████

W1104 17:14:18.838911 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/note_sequence_io.py:60: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W1104 17:14:18.842164 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/pipelines/statistics.py:132: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W1104 17:14:20.483544 139944538335104 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Done


In [4]:
#@title Load the pre-trained model.
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_2bar_big.ckpt')

W1104 17:14:24.342042 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/models/music_vae/trained_model.py:58: The name tf.gfile.IsDirectory is deprecated. Please use tf.io.gfile.isdir instead.

W1104 17:14:24.352546 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/models/music_vae/base_model.py:159: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.

W1104 17:14:24.363215 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/models/music_vae/lstm_models.py:104: The name tf.VariableScope is deprecated. Please use tf.compat.v1.VariableScope instead.

W1104 17:14:24.370161 139944538335104 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/magenta/models/music_vae/lstm_utils.py:44: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.


In [0]:
genValues = [[0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.2, 0.7, 0.6, 0.0, 0.2, 0.3, 0.1, 0.3, 0.9, 0.2, 0.5, 0.0, 0.3, 0.1, 0.3, 0.9, 0.6, 0.7, 0.4, 0.7, 0.5, 0.1, 0.0, 0.7, 0.5, 0.7, 0.2, 0.0, 0.1, 0.1, 0.8, 0.2, 0.0, 0.3, 0.6, 0.0, 0.6, 0.5, 0.0, 0.7, 0.8, 0.4, 0.3, 0.7, 0.7, 0.8, 0.5, 0.4, 0.7, 0.7, 0.7, 0.1, 0.5, 0.1, 0.7, 0.7, 0.4, 0.0, 0.4, 0.1, 0.5, 0.1, 0.6, 0.4, 0.9, 0.3, 0.5, 0.2, 0.0, 0.9, 0.8, 0.6, 0.5, 0.6, 0.5, 0.4, 0.8, 0.0, 0.4, 0.3, 0.3, 0.1, 0.0, 0.7, 0.6, 0.9, 0.5, 0.1, 0.0, 0.6, 0.1, 0.2, 0.1, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.6, 0.0, 0.3, 0.5, 0.5, 0.5, 0.9, 0.3, 0.6, 0.5, 0.0, 0.0, 0.0, 0.1, 0.1, 0.4, 0.6, 0.6, 0.2, 0.4, 0.0, 0.7, 0.3, 0.2, 0.9, 0.5, 0.4, 0.2, 0.7, 0.9, 0.2, 0.0, 0.7, 0.7, 0.7, 0.7, 0.6, 0.6, 0.2, 0.6, 0.0, 0.2, 0.9, 0.9, 0.5, 0.9, 0.2, 0.9, 0.2, 0.8, 0.8, 0.9, 0.1, 0.0, 0.7, 0.7, 0.2, 0.5, 0.7, 0.6, 0.2, 0.8, 0.3, 0.9, 0.9, 0.5, 0.1, 0.6, 0.0, 0.9, 0.7, 0.7, 0.2, 0.1, 0.2, 0.7, 0.9, 0.4, 0.8, 0.5, 0.9, 0.7, 0.3, 0.0, 0.3, 0.9, 0.9, 0.2, 0.7, 0.5, 0.8, 0.4, 0.3, 0.7, 0.4, 0.7, 0.1, 0.5, 0.7, 0.3, 0.9, 0.9, 0.7, 0.8, 0.5, 0.1, 0.7, 0.7, 0.7, 0.7, 0.5, 0.2, 0.4, 0.8, 0.1, 0.2, 0.8, 0.4, 0.5, 0.7, 0.9, 0.8, 0.4, 0.0, 0.1, 0.3, 0.8, 0.7, 0.6, 0.0, 0.5, 0.6, 0.5, 0.9, 0.4, 0.6, 0.1, 0.6, 0.4, 0.4, 0.8, 0.6, 0.1, 0.5, 0.2, 0.7, 0.8, 0.8, 0.1, 0.4, 0.2, 0.4, 0.3, 0.6, 0.3, 0.4, 0.9, 0.3, 0.7, 0.6, 0.0, 0.4, 0.4, 0.6, 0.8, 0.5, 0.7, 0.1, 0.5, 0.4, 0.4, 0.5, 0.6, 0.0, 0.9, 0.9, 0.6, 0.7, 0.1, 0.3, 0.7, 0.1, 0.3, 0.0, 0.0, 0.2, 0.3, 0.3, 0.1, 0.2, 0.4, 0.1, 0.8, 0.4, 0.8, 0.5, 0.4, 0.3, 0.3, 0.5, 0.2, 0.0, 0.1, 0.7, 0.7, 0.0, 0.4, 0.6, 0.1, 0.1, 0.9, 0.6, 0.2, 0.2, 0.1, 0.4, 0.7, 0.1, 0.1, 0.8, 0.0, 0.3, 0.3, 0.9, 0.3, 0.2, 0.4, 0.3, 0.3, 0.9, 0.7, 0.8, 0.6, 0.8, 0.3, 0.2, 0.8, 0.0, 0.9, 0.1, 0.7, 0.3, 0.0, 0.8, 0.2, 0.9, 0.2, 0.4, 0.5, 0.0, 0.0, 0.5, 0.6, 0.7, 0.5, 0.3, 0.5, 0.2, 0.1, 0.0, 0.8, 0.4, 0.0, 0.7, 0.0, 0.0, 0.4, 0.7, 0.1, 0.3, 0.4, 0.9, 0.2, 0.2, 0.0, 0.5, 0.4, 0.7, 0.0, 0.7, 0.8, 0.9, 0.3, 0.6, 0.8, 0.4, 0.5, 0.0, 0.2, 0.1, 0.3, 0.4, 0.7, 0.0, 0.8, 0.0, 0.0, 0.9, 0.9, 0.4, 0.6, 0.6, 0.7, 0.9, 0.6, 0.7, 0.1, 0.5, 0.5, 0.6, 0.4, 0.5, 0.3, 0.7, 0.6, 0.1, 0.9, 0.7, 0.7, 0.5, 0.1, 0.4, 0.8, 0.6, 0.4, 0.4, 0.4, 0.4, 0.7, 0.7, 0.2, 0.2, 0.8, 0.6, 0.3, 0.3, 0.5, 0.2, 0.1, 0.1, 0.3, 0.3, 0.0, 0.3, 0.8, 0.4, 0.0, 0.9, 0.5, 0.5, 0.0, 0.6, 0.2, 0.7, 0.0, 0.3, 0.7, 0.7, 0.6, 0.6, 0.3, 0.9, 0.4, 0.7, 0.8, 0.5, 0.0, 0.8, 0.5, 0.9, 0.1, 0.9, 0.6, 0.8, 0.8], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.2, 0.7, 0.6, 0.0, 0.2, 0.3, 0.1, 0.3, 0.9, 0.2, 0.5, 0.0, 0.3, 0.1, 0.3, 0.9, 0.6, 0.7, 0.4, 0.7, 0.5, 0.1, 0.0, 0.7, 0.5, 0.7, 0.2, 0.0, 0.1, 0.1, 0.5, 0.6, 0.9, 0.2, 0.5, 0.1, 0.8, 0.0, 0.1, 0.4, 0.4, 0.2, 0.5, 0.5, 0.9, 0.9, 0.9, 0.2, 0.3, 0.4, 0.6, 0.7, 0.4, 0.7, 0.1, 0.7, 0.5, 0.7, 0.6, 0.5, 0.1, 0.5, 0.0, 0.0, 0.7, 0.8, 0.1, 0.3, 0.6, 0.8, 0.1, 0.3, 0.1, 0.3, 0.4, 0.3, 0.1, 0.4, 0.3, 0.1, 0.9, 0.8, 0.5, 0.3, 0.4, 0.5, 0.3, 0.6, 0.8, 0.5, 0.5, 0.5, 0.7, 0.2, 0.5, 0.9, 0.1, 0.0, 0.3, 0.8, 0.5, 0.7, 0.9, 0.4, 0.5, 0.8, 0.1, 0.5, 0.4, 0.5, 0.3, 0.2, 0.7, 0.0, 0.5, 0.0, 0.3, 0.8, 0.0, 0.7, 0.6, 0.1, 0.5, 0.3, 0.8, 0.8, 0.7, 0.2, 0.1, 0.7, 0.0, 0.4, 0.7, 0.7, 0.7, 0.0, 0.3, 0.8, 0.9, 0.9, 0.7, 0.9, 0.6, 0.3, 0.0, 0.7, 0.6, 0.4, 0.1, 0.0, 0.2, 0.8, 0.6, 0.2, 0.1, 0.4, 0.2, 0.4, 0.6, 0.8, 0.1, 0.9, 0.6, 0.4, 0.2, 0.3, 0.1, 0.7, 0.0, 0.9, 0.3, 0.0, 0.9, 0.0, 0.6, 0.6, 0.3, 0.8, 0.2, 0.5, 0.0, 0.8, 0.7, 0.2, 0.8, 0.1, 0.2, 0.9, 0.3, 0.0, 0.9, 0.0, 0.4, 0.5, 0.0, 0.9, 0.8, 0.3, 0.2, 0.5, 0.0, 0.2, 0.9, 0.2, 0.6, 0.9, 0.7, 0.5, 0.4, 0.7, 0.5, 0.9, 0.2, 0.8, 0.2, 0.6, 0.6, 0.3, 0.7, 0.1, 0.0, 0.8, 0.4, 0.4, 0.7, 0.5, 0.6, 0.4, 0.1, 0.2, 0.5, 0.4, 0.0, 0.6, 0.8, 0.8, 0.7, 0.5, 0.5, 0.1, 0.4, 0.8, 0.3, 0.7, 0.2, 0.2, 0.6, 0.4, 0.2, 0.9, 0.5, 0.2, 0.5, 0.1, 0.9, 0.7, 0.7, 0.1, 0.4, 0.5, 0.6, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.3, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.0, 0.0, 0.7, 0.3, 0.7, 0.4, 0.4, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.2, 0.7, 0.6, 0.0, 0.2, 0.3, 0.1, 0.3, 0.9, 0.2, 0.5, 0.0, 0.3, 0.1, 0.3, 0.9, 0.6, 0.7, 0.4, 0.7, 0.5, 0.1, 0.0, 0.7, 0.5, 0.7, 0.2, 0.0, 0.1, 0.1, 0.5, 0.6, 0.9, 0.2, 0.5, 0.1, 0.8, 0.0, 0.1, 0.4, 0.4, 0.2, 0.5, 0.5, 0.9, 0.9, 0.9, 0.2, 0.3, 0.4, 0.6, 0.7, 0.4, 0.7, 0.1, 0.7, 0.5, 0.7, 0.6, 0.5, 0.1, 0.5, 0.0, 0.0, 0.7, 0.8, 0.1, 0.3, 0.6, 0.8, 0.1, 0.3, 0.1, 0.3, 0.4, 0.3, 0.1, 0.4, 0.3, 0.1, 0.9, 0.8, 0.5, 0.3, 0.4, 0.5, 0.3, 0.6, 0.8, 0.5, 0.5, 0.5, 0.7, 0.2, 0.5, 0.9, 0.1, 0.0, 0.3, 0.8, 0.5, 0.7, 0.9, 0.4, 0.5, 0.8, 0.1, 0.5, 0.4, 0.5, 0.3, 0.2, 0.7, 0.0, 0.5, 0.0, 0.3, 0.8, 0.0, 0.7, 0.6, 0.1, 0.5, 0.3, 0.8, 0.8, 0.7, 0.2, 0.1, 0.7, 0.0, 0.4, 0.7, 0.7, 0.7, 0.0, 0.3, 0.8, 0.9, 0.9, 0.7, 0.9, 0.6, 0.3, 0.0, 0.7, 0.6, 0.4, 0.1, 0.0, 0.2, 0.8, 0.6, 0.2, 0.1, 0.4, 0.2, 0.4, 0.6, 0.8, 0.1, 0.9, 0.6, 0.4, 0.2, 0.3, 0.1, 0.7, 0.0, 0.9, 0.3, 0.0, 0.9, 0.0, 0.6, 0.6, 0.3, 0.8, 0.2, 0.5, 0.0, 0.8, 0.7, 0.2, 0.8, 0.1, 0.2, 0.9, 0.3, 0.0, 0.9, 0.0, 0.4, 0.5, 0.0, 0.9, 0.8, 0.3, 0.2, 0.5, 0.0, 0.2, 0.9, 0.2, 0.6, 0.9, 0.7, 0.5, 0.4, 0.7, 0.5, 0.9, 0.2, 0.8, 0.2, 0.6, 0.6, 0.3, 0.7, 0.1, 0.0, 0.8, 0.4, 0.4, 0.7, 0.5, 0.6, 0.4, 0.1, 0.2, 0.5, 0.4, 0.0, 0.6, 0.8, 0.8, 0.7, 0.5, 0.5, 0.1, 0.4, 0.8, 0.3, 0.7, 0.2, 0.2, 0.6, 0.4, 0.2, 0.9, 0.5, 0.2, 0.5, 0.1, 0.9, 0.7, 0.7, 0.1, 0.4, 0.5, 0.6, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.3, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.0, 0.0, 0.7, 0.3, 0.7, 0.4, 0.4, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.2, 0.7, 0.6, 0.0, 0.2, 0.3, 0.1, 0.3, 0.9, 0.2, 0.5, 0.0, 0.3, 0.1, 0.3, 0.9, 0.6, 0.7, 0.4, 0.7, 0.5, 0.1, 0.0, 0.7, 0.5, 0.7, 0.2, 0.0, 0.1, 0.1, 0.5, 0.6, 0.9, 0.2, 0.5, 0.1, 0.8, 0.0, 0.1, 0.4, 0.4, 0.2, 0.5, 0.5, 0.9, 0.9, 0.9, 0.2, 0.3, 0.4, 0.6, 0.7, 0.4, 0.7, 0.1, 0.7, 0.5, 0.7, 0.6, 0.5, 0.1, 0.5, 0.0, 0.0, 0.7, 0.8, 0.1, 0.3, 0.6, 0.8, 0.1, 0.3, 0.1, 0.3, 0.4, 0.3, 0.1, 0.4, 0.3, 0.1, 0.9, 0.8, 0.5, 0.3, 0.4, 0.5, 0.3, 0.6, 0.8, 0.5, 0.5, 0.5, 0.7, 0.2, 0.5, 0.9, 0.1, 0.0, 0.3, 0.8, 0.5, 0.7, 0.9, 0.4, 0.5, 0.8, 0.1, 0.5, 0.4, 0.5, 0.3, 0.2, 0.7, 0.0, 0.5, 0.0, 0.3, 0.8, 0.0, 0.7, 0.6, 0.1, 0.5, 0.3, 0.8, 0.8, 0.7, 0.2, 0.1, 0.7, 0.0, 0.4, 0.7, 0.7, 0.7, 0.0, 0.3, 0.8, 0.9, 0.9, 0.7, 0.9, 0.6, 0.3, 0.0, 0.7, 0.6, 0.4, 0.1, 0.0, 0.2, 0.8, 0.6, 0.2, 0.1, 0.4, 0.2, 0.4, 0.6, 0.8, 0.1, 0.9, 0.6, 0.4, 0.2, 0.3, 0.1, 0.7, 0.0, 0.9, 0.3, 0.0, 0.9, 0.0, 0.6, 0.6, 0.3, 0.8, 0.2, 0.5, 0.0, 0.8, 0.7, 0.2, 0.8, 0.1, 0.2, 0.9, 0.3, 0.0, 0.9, 0.0, 0.4, 0.5, 0.0, 0.9, 0.8, 0.3, 0.2, 0.5, 0.0, 0.2, 0.9, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.3, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.2, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.2, 0.5, 0.3, 0.1, 0.4, 0.1, 0.3, 0.1, 0.2, 0.8, 0.6, 0.2, 0.7, 0.9, 0.9, 0.0, 0.8, 0.2, 0.0, 0.8, 0.4, 0.5, 0.9, 0.8, 0.6, 0.4, 0.3, 0.8, 0.2, 0.9, 0.6, 0.9, 0.0, 0.4, 0.1, 0.7, 0.8, 0.5, 0.5, 0.6, 0.6, 0.7, 0.6, 0.4, 0.2, 0.1, 0.3, 0.8, 0.7, 0.2, 0.7, 0.5, 0.7, 0.0, 0.5, 0.7, 0.3, 0.5, 0.9, 0.3, 0.8, 0.5, 0.6, 0.3, 0.6, 0.3, 0.2, 0.5, 0.9, 0.7, 0.4, 0.1, 0.2, 0.0, 0.3, 0.6, 0.6, 0.6, 0.5, 0.5, 0.0, 0.3, 0.8, 0.9, 0.5, 0.2, 0.5, 0.0, 0.3, 0.2, 0.3, 0.3, 0.0, 0.9, 0.9, 0.4, 0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.7, 0.0, 0.6, 0.5, 0.5, 0.8, 0.6, 0.3, 0.6, 0.2, 0.2, 0.9, 0.2, 0.1, 0.2, 0.1, 0.6, 0.8, 0.3, 0.9, 0.5, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.3, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.2, 0.5, 0.3, 0.1, 0.4, 0.1, 0.3, 0.1, 0.2, 0.8, 0.6, 0.2, 0.7, 0.9, 0.9, 0.0, 0.8, 0.2, 0.0, 0.8, 0.4, 0.5, 0.9, 0.8, 0.6, 0.4, 0.3, 0.8, 0.2, 0.9, 0.6, 0.9, 0.0, 0.4, 0.1, 0.7, 0.8, 0.5, 0.5, 0.6, 0.6, 0.7, 0.6, 0.4, 0.2, 0.1, 0.3, 0.8, 0.7, 0.2, 0.7, 0.5, 0.7, 0.0, 0.5, 0.7, 0.3, 0.5, 0.9, 0.3, 0.8, 0.5, 0.6, 0.3, 0.6, 0.3, 0.2, 0.5, 0.9, 0.7, 0.4, 0.1, 0.2, 0.0, 0.3, 0.6, 0.6, 0.6, 0.5, 0.5, 0.0, 0.3, 0.8, 0.9, 0.5, 0.2, 0.5, 0.0, 0.3, 0.2, 0.3, 0.3, 0.0, 0.9, 0.9, 0.4, 0.0, 0.0, 0.0, 0.0, 0.1, 0.2, 0.7, 0.0, 0.6, 0.5, 0.5, 0.8, 0.6, 0.3, 0.6, 0.2, 0.2, 0.9, 0.2, 0.1, 0.2, 0.1, 0.6, 0.8, 0.3, 0.9, 0.5, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.3, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.2, 0.5, 0.3, 0.1, 0.4, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.3, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.6, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.2, 0.5, 0.3, 0.1, 0.4, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.3, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.6, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.2, 0.5, 0.3, 0.1, 0.4, 0.1, 0.3, 0.1, 0.2, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.3, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.6, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.7, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.5, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.5, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.9, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.2, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.8, 0.1, 0.6, 0.3, 0.9, 0.8, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.9, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.4, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.4, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.1, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.4, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.2, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.4, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.5, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.2, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.0, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.6, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.3, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.2, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.6, 0.5, 0.1, 0.7, 0.5, 0.8, 0.0, 0.1, 0.2, 0.1, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.7, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.1, 0.8, 0.8, 0.2, 0.7, 0.8, 0.5, 0.4, 0.1, 0.2, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.4, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.7, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.0, 0.8, 0.4, 0.2, 0.2, 0.3, 0.1, 0.4, 0.6, 0.7, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.2, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.6, 0.5, 0.7, 0.5, 0.9, 0.4, 0.2, 0.1, 0.7, 0.5, 0.4, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.3, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.0, 0.8, 0.4, 0.2, 0.2, 0.3, 0.1, 0.4, 0.6, 0.7, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.2, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.6, 0.5, 0.7, 0.5, 0.9, 0.4, 0.2, 0.1, 0.7, 0.5, 0.4, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.7, 0.3, 0.6, 0.7, 0.4, 0.2, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.0, 0.8, 0.4, 0.2, 0.2, 0.3, 0.1, 0.4, 0.6, 0.7, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.2, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.2, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.6, 0.5, 0.7, 0.5, 0.9, 0.4, 0.2, 0.1, 0.7, 0.5, 0.4, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.6, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.8, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.1, 0.3, 0.6, 0.7, 0.4, 0.1, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.2, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.0, 0.8, 0.4, 0.2, 0.2, 0.3, 0.1, 0.4, 0.6, 0.7, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.2, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.9, 0.8, 0.9, 0.6, 0.6, 0.5, 0.7, 0.5, 0.9, 0.4, 0.2, 0.1, 0.7, 0.5, 0.4, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.9, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.5, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.9, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.1, 0.3, 0.6, 0.7, 0.4, 0.1, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.2, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.6, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4], [0.3, 0.8, 0.1, 0.1, 0.3, 0.8, 0.9, 0.0, 0.9, 0.3, 0.2, 0.9, 0.5, 0.5, 0.3, 0.8, 0.0, 0.0, 0.5, 0.9, 0.6, 0.6, 0.1, 0.4, 0.3, 0.9, 0.1, 0.7, 0.3, 0.1, 0.8, 0.0, 0.3, 0.4, 0.2, 0.3, 0.9, 0.9, 0.6, 0.7, 0.2, 0.1, 0.8, 0.7, 0.3, 0.1, 0.8, 0.2, 0.4, 0.0, 0.0, 0.0, 0.8, 0.4, 0.2, 0.2, 0.3, 0.1, 0.4, 0.6, 0.7, 0.2, 0.4, 0.5, 0.5, 0.9, 0.4, 0.0, 0.4, 0.9, 0.5, 0.3, 0.3, 0.3, 0.7, 0.6, 0.5, 0.3, 0.2, 0.5, 0.7, 0.4, 0.8, 0.4, 0.7, 0.4, 0.5, 0.7, 0.1, 0.0, 0.4, 0.6, 0.6, 0.9, 0.1, 0.4, 0.3, 0.3, 0.5, 0.9, 0.0, 0.0, 0.5, 0.4, 0.5, 0.1, 0.0, 0.8, 0.1, 0.9, 0.2, 0.6, 0.1, 0.9, 0.2, 0.4, 0.0, 0.9, 0.0, 0.7, 0.1, 0.1, 0.1, 0.6, 0.0, 0.1, 0.7, 0.5, 0.1, 0.2, 0.3, 0.7, 0.0, 0.5, 0.2, 0.5, 0.4, 0.6, 0.9, 0.2, 0.6, 0.5, 0.7, 0.4, 0.3, 0.5, 0.7, 0.6, 0.2, 0.9, 0.5, 0.8, 0.9, 0.6, 0.6, 0.5, 0.7, 0.5, 0.9, 0.4, 0.2, 0.1, 0.7, 0.5, 0.4, 0.2, 0.0, 0.0, 0.8, 0.0, 0.3, 0.2, 0.2, 0.7, 0.8, 0.5, 0.1, 0.2, 0.8, 0.8, 0.4, 0.4, 0.9, 0.2, 0.0, 0.2, 0.9, 0.6, 0.3, 0.0, 0.6, 0.1, 0.3, 0.9, 0.9, 0.6, 0.4, 0.9, 0.6, 0.8, 0.0, 0.0, 0.1, 0.7, 0.5, 0.9, 0.1, 0.3, 0.3, 0.7, 0.4, 0.4, 0.1, 0.6, 0.0, 0.4, 0.5, 0.1, 0.8, 0.7, 0.5, 0.1, 0.9, 0.9, 0.1, 0.9, 0.1, 0.4, 0.4, 0.3, 0.3, 0.9, 0.1, 0.3, 0.4, 0.1, 0.0, 0.4, 0.5, 0.2, 0.8, 0.6, 0.2, 0.5, 0.0, 0.1, 0.8, 0.2, 0.4, 0.1, 0.1, 0.8, 0.8, 0.4, 0.4, 0.4, 0.1, 0.1, 0.2, 0.0, 0.8, 0.5, 0.0, 0.9, 0.7, 0.3, 0.8, 0.2, 0.3, 0.6, 0.2, 0.9, 0.9, 0.5, 0.7, 0.6, 0.5, 0.7, 0.4, 0.7, 0.3, 0.8, 0.0, 0.1, 0.4, 0.4, 0.8, 0.8, 0.9, 0.8, 0.6, 0.4, 0.8, 0.9, 0.1, 0.9, 0.9, 0.0, 0.0, 0.6, 0.6, 0.5, 0.6, 0.5, 0.6, 0.7, 0.6, 0.2, 0.8, 0.8, 0.8, 0.6, 0.2, 0.1, 0.0, 0.4, 0.0, 0.1, 0.2, 0.3, 0.2, 0.7, 0.3, 0.3, 0.6, 0.6, 0.9, 0.4, 0.3, 0.5, 0.4, 0.1, 0.0, 0.7, 0.5, 0.5, 0.7, 0.9, 0.4, 0.6, 0.2, 0.7, 0.2, 0.5, 0.0, 0.0, 0.4, 0.2, 0.1, 0.4, 0.9, 0.9, 0.0, 0.7, 0.2, 0.3, 0.0, 0.7, 0.4, 0.8, 0.7, 0.4, 0.4, 0.3, 0.2, 0.1, 0.6, 0.1, 0.8, 0.3, 0.1, 0.3, 0.6, 0.7, 0.4, 0.1, 0.9, 0.2, 0.2, 0.2, 0.5, 0.2, 0.5, 0.9, 0.2, 0.2, 0.9, 0.4, 0.2, 0.6, 0.4, 0.2, 0.3, 0.5, 0.0, 0.2, 0.4, 0.4, 0.8, 0.3, 0.2, 0.7, 0.9, 0.6, 0.2, 0.2, 0.5, 0.3, 0.4, 0.5, 0.4, 0.3, 0.7, 0.3, 0.2, 0.6, 0.7, 0.5, 0.8, 0.2, 0.2, 0.1, 0.3, 0.1, 0.3, 0.6, 0.1, 0.5, 0.3, 0.0, 0.5, 0.0, 0.8, 0.5, 0.4, 0.2, 0.5, 0.2, 0.4, 0.7, 0.4, 0.1, 0.1, 0.6, 0.0, 0.3, 0.5, 0.9, 0.7, 0.0, 0.9, 0.0, 0.7, 0.3, 0.7, 0.5, 0.5, 0.5, 0.1, 0.9, 0.1, 0.2, 0.2, 0.1, 0.6, 0.3, 0.9, 0.6, 0.0, 0.5, 0.2, 0.2, 0.6, 0.0, 0.6, 0.2, 0.5, 0.1, 0.0, 0.2, 0.4, 0.6, 0.5, 0.3, 0.0, 0.2, 0.9, 0.9, 0.0, 0.6, 0.7, 0.6, 0.3, 0.2, 0.0, 0.6, 0.1, 0.3, 0.3, 0.3, 0.5, 0.8, 0.4, 0.2, 0.3, 0.7, 0.8, 0.3, 0.6, 0.3, 0.0, 0.4]]

In [7]:
#@title Generate samples from the prior.

seq = mel_2bar.decode(genValues, 32)
for ns in seq:
  play(ns)

In [8]:
for i, ns in enumerate(seq):
  download(ns, 'mel_2bar_GA_%d.mid' % i)

W1104 17:22:46.913433 139944538335104 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/magenta/music/midi_io.py:220: The name tf.gfile.Copy is deprecated. Please use tf.io.gfile.copy instead.

